In [85]:
%run utils.ipynb

In [86]:
hb_df = pd.read_csv(dataset_path, sep=';').drop(['Unnamed: 0', 'patient_id'], axis=1)

In [87]:
# Fill missing dataset in the dataset
hb_df, _ = knn_impute_missing_data(hb_df)
hb_df, _ = round_columns_to_allowed_values(hb_df, 
                                           columns_to_round= ['alcohol_scale', 'diary_mood_avg', 'diary_sleep_avg', 'diary_stress_avg'])

In [88]:
# Features engineering
## MASS is the abreviation of Mood Alcohol Stress Sleep
hb_df['mass_score'] = hb_df.apply(lambda r: create_mass_score(r), axis=1)
hb_df['combined_age_medical_condition'] = (hb_df['age'] * 
                                           (1 + 0.5 * hb_df['pre_existing_medical_condition'].apply(lambda v: 1 if v=='yes' else 0)))
hb_df['combined_to_assessment_age'] = (hb_df['to_assessment'] * (1 - 0.01 * hb_df['age']))


In [89]:
# Transform columns
hb_df['age'] = np.log(hb_df['age'] + 1)
hb_df['to_assessment'] = np.log(hb_df['to_assessment'] + 1)
hb_df['combined_age_medical_condition'] = np.log(hb_df['combined_age_medical_condition'] + 1)
hb_df['combined_to_assessment_age'] = np.log(hb_df['combined_to_assessment_age'] + 1)

In [90]:
#hb_df = hb_df[['to_assessment', 'course_burnout', 'course_chronic_pain', 'course_depression', 'course_sleep', 'course_stress', 'gender', 'age', 'pre_existing_medical_condition', 'mass_score', 'unsuccessful_outcome']]
#hb_df = hb_df[['to_assessment', 'course', 'gender', 'age', 'pre_existing_medical_condition', 'mass_score', 'unsuccessful_outcome']]
hb_df = hb_df[['combined_to_assessment_age', 'course', 'gender', 'combined_age_medical_condition', 'mass_score', 'unsuccessful_outcome']]

In [91]:
hb_df

,combined_to_assessment_age,course,gender,combined_age_medical_condition,mass_score,unsuccessful_outcome
0,0.091996,depression,male,3.761200,1.1,0.0
1,0.017870,depression,male,4.350278,2.8,0.0
2,0.000124,depression,female,3.850148,3.8,0.0
3,1.457414,sleep,other,4.248495,2.1,0.0
4,0.150624,chronic_pain,female,4.290459,3.0,0.0
...,...,...,...,...,...,...
9995,2.067951,stress,female,3.979682,4.2,0.0
9996,0.776884,depression,female,4.620059,2.7,0.0
9997,1.940019,sleep,other,3.481240,1.4,0.0
9998,1.939029,chronic_pain,other,4.204693,4.6,0.0


In [92]:
# Save to parcquet file
hb_df.to_parquet(processed_dataset_path)

In [93]:
hb_df.describe().T

,count,mean,std,min,25%,50%,75%,max
combined_to_assessment_age,10000.0,0.646732,0.809954,0.000000,0.004807,0.205512,1.182606,2.856441
combined_age_medical_condition,10000.0,3.941467,0.425530,2.944439,3.637586,3.988984,4.248495,4.677491
mass_score,10000.0,2.515890,0.903715,0.000000,1.900000,2.500000,3.200000,5.000000
unsuccessful_outcome,10000.0,0.233100,0.422827,0.000000,0.000000,0.000000,0.000000,1.000000
